In [47]:
import json
import warnings
import requests
import tqdm

In [48]:
#Load your api key
with open('api_keys.json') as f:
    api_keys = json.load(f)
alt_metric_key = api_keys['alt_metric']
def get_reduced_alt_values(r):
    """when some info is missing extracts less information from the response object"""

    score = r['score']
    try:
        percentile = r['altmetric_score']['context_for_score']['all']['percentile']
    except:
        percentile = float('NaN')
    try:
        similar_age_3m_percentile = r['altmetric_score']['context_for_score']['similar_age_3m']
    except:
        similar_age_3m_percentile = float('NaN')
    try:
        total_readers = 0
        for key in r['counts']['readers'].keys():
            total_readers += int(r['counts']['readers'][key])
    except:
        total_readers = float('NaN')
    alt_values = {'score': score}, {'percentile': percentile}, {'similar_age_3m_percentile': similar_age_3m_percentile}, {'total_readers': total_readers}
    return alt_values
def get_alt_values(r):
    """extracts relevant information from the response object"""
    total_readers = 0
    for key in r['counts']['readers'].keys():
        total_readers += int(r['counts']['readers'][key])
    score = r['score']
    percentile = r['altmetric_score']['context_for_score']['all']['percentile']
    similar_age_3m_percentile= r['altmetric_score']['context_for_score']['similar_age_3m']  #['percentile']
    alt_values = {'score': score}, {'percentile': percentile}, {'similar_age_3m_percentile': similar_age_3m_percentile}, {'total_readers': total_readers}
    return alt_values
def altmetric_arxiv_requests(arxiv):
    """make request with arxiv id"""
    global alt_metric_key
    r = requests.get(f'https://api.altmetric.com/v1/fetch/arxiv_id/{arxiv}?key={alt_metric_key}')
    if r.status_code != 200:
        print('Warning - Response Code ', r.status_code)
    return r.json()
def altmetric_doi_requests(doi):
    """make request with arxiv doi"""
    global alt_metric_key
    r = requests.get(f'https://api.altmetric.com/v1/fetch/doi/{doi}?key={alt_metric_key}')
    if r.status_code != 200:
        print('Warning - Response Code ', r.status_code, 'DOI', doi)
    return r.json()
altmetric_doi_requests('10.1177%2F0278364913491297')['altmetric_score']['context_for_score']['all']

{'total_number_of_other_articles': 22796179,
 'mean': 10.19502521226795,
 'rank': 2415136,
 'this_scored_higher_than_pct': 89,
 'this_scored_higher_than': 20369550,
 'rank_type': 'exact',
 'sample_size': 22796179,
 'percentile': 89}

In [49]:
suffix = '_22_12_2022'
file_name = f'data/data_sorted_only_w_ids{suffix}.json'
with open(file_name, "r") as ds:
    retrieved_data = json.load(ds)
print('Number of Datasets:', len(retrieved_data))
c = 0
c_doi = 0
for idx_paper, paper in enumerate(retrieved_data):
    c += 1
    try:
        doi = paper['DOI']
        r = altmetric_doi_requests(doi)
        c_doi += 1
    except:
        try:
            r = altmetric_arxiv_requests(paper['arxivId'])
            c_doi += 1
        except Exception as ex:
            warnings.warn(f'{paper["id"]} reqeust caused: {ex}')
        continue
    try:
        alt_values = get_alt_values(r)
    except:
        try:
            alt_values = get_reduced_alt_values(r)
        except Exception as exec:
            print(exec)
            c_doi -= 1
    retrieved_data[idx_paper].update({'altmetrics': alt_values})
#with open(f'data/data_sorted_only_w_ids{suffix}_altmetrics.json', 'w', encoding='utf-8') as f:
#    json.dump(retrieved_data, f)
print('found:', c_doi, '/', c)

Number of Datasets: 186
Warning - Response Code  404 DOI 10.1109/IJCNN.2013.6706807


C:\Users\jonas\AppData\Local\Temp\ipykernel_9376\3423221373.py:19: UserWarning: GTSDB caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')


Warning - Response Code  404 DOI 10.1177/0278364913507326
Warning - Response Code  404 DOI -


C:\Users\jonas\AppData\Local\Temp\ipykernel_9376\3423221373.py:19: UserWarning: Malaga Stereo and Laser Urban caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')


Warning - Response Code  404 DOI 10.48550/arXiv.1608.01230
Warning - Response Code  404 DOI 10.1109/ICRA.2011.5979636


...

Warning - Response Code  404 DOI 10.1109/ICASSP43922.2022.9747779


C:\Users\jonas\AppData\Local\Temp\ipykernel_9376\3423221373.py:19: UserWarning: HDBD caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')


Warning - Response Code  404 DOI 10.1109/IROS40897.2019.8968283
Warning - Response Code  404 DOI 10.48550/arXiv.1712.02463


C:\Users\jonas\AppData\Local\Temp\ipykernel_9376\3423221373.py:19: UserWarning: RUGD: Robot Unstructured Ground Driving caused: 'arxivId'
  warnings.warn(f'{paper["id"]} caused: {ex}')


found: 159 / 186


In [63]:
zero_counter = 0
for data in retrieved_data:
    try:
        perc = data['altmetrics'][1]['percentile']
        if perc >= 1:
            zero_counter += 1
    except:
        continue
zero_counter

26

In [51]:
doi = '10.1177%2F0278364913491297'
r = requests.get(f'https://api.altmetric.com/v1/fetch/doi/{doi}?key={alt_metric_key}')

In [57]:
c = 0
for paper in retrieved_data:
    try:
        year = int(paper['publishDate'][:4])

    except:
        continue
    #print(year)
    if year == 2022:
        try:
            print(paper['id'], paper['altmetrics'], '\n')
            c += 1
        except:
            continue

KITTI-360 PanopticBEV ({'score': 1}, {'percentile': 22}, {'similar_age_3m_percentile': {'total_number_of_other_articles': 470716, 'mean': 15.343056981278384, 'rank': 272899, 'this_scored_higher_than_pct': 31, 'this_scored_higher_than': 147657, 'rank_type': 'exact', 'sample_size': 470716, 'percentile': 31}}, {'total_readers': 64}) 

nuScenes PanopticBEV ({'score': 1}, {'percentile': 22}, {'similar_age_3m_percentile': {'total_number_of_other_articles': 470716, 'mean': 15.343056981278384, 'rank': 272899, 'this_scored_higher_than_pct': 31, 'this_scored_higher_than': 147657, 'rank_type': 'exact', 'sample_size': 470716, 'percentile': 31}}, {'total_readers': 64}) 

ScribbleKITTI ({'score': 0}, {'percentile': nan}, {'similar_age_3m_percentile': nan}, {'total_readers': 0}) 

SHIFT ({'score': 0}, {'percentile': nan}, {'similar_age_3m_percentile': nan}, {'total_readers': 63}) 

CrashD ({'score': 0}, {'percentile': nan}, {'similar_age_3m_percentile': nan}, {'total_readers': 30}) 

Rope3D ({'score'

In [53]:
import pandas as pd
test = pd.Series([-100, -100, 0, 2, 3])

In [54]:
test.rank(pct=True)

0    0.3
1    0.3
2    0.6
3    0.8
4    1.0
dtype: float64

In [55]:
test.rank(pct=True).rank(pct=True)

0    0.3
1    0.3
2    0.6
3    0.8
4    1.0
dtype: float64